![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Project Overview: Redis University Class Agent

## Learning Objectives (30 minutes)
By the end of this notebook, you will be able to:
1. **Describe** the architecture of the Redis University Class Agent
2. **Identify** the key components: LangGraph, Redis, Agent Memory Server, OpenAI
3. **Explain** how the reference agent demonstrates context engineering principles
4. **Navigate** the project structure and understand the codebase organization
5. **Run** basic agent interactions and understand the workflow

## Prerequisites
- Completed "01_what_is_context_engineering.ipynb"
- Basic understanding of AI agents and language models
- Environment setup (covered in next notebook: "03_setup_environment.ipynb")

---

## Introduction

Throughout this course, we'll be building and exploring a complete **Redis University Class Agent** - a sophisticated AI agent that helps students find courses, plan their academic journey, and provides personalized recommendations.

This project serves as a comprehensive example of context engineering principles in action, demonstrating how to build intelligent, context-aware AI systems using Redis, LangGraph, and modern AI tools.

## Project Goals

Our Redis University Class Agent is designed to:

### 🎯 **Primary Objectives**
- **Help students discover relevant courses** based on their interests and goals
- **Provide personalized recommendations** considering academic history and preferences
- **Remember student context** across multiple conversations and sessions
- **Answer questions** about courses, prerequisites, and academic planning
- **Adapt and learn** from student interactions over time

### 📚 **Educational Objectives**
- **Demonstrate context engineering concepts** in a real-world scenario
- **Show Redis capabilities** for AI applications and memory management
- **Illustrate LangGraph workflows** for complex agent behaviors
- **Provide a reference implementation** for similar projects
- **Teach best practices** for building context-aware AI systems

## System Architecture

Our agent follows a modern, scalable architecture:

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   User Input    │───▶│   LangGraph     │───▶│   OpenAI GPT    │
│   (CLI/API)     │    │   Agent         │    │   (LLM)         │
└─────────────────┘    └─────────────────┘    └─────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    Redis Context Engine                         │
├─────────────────┬─────────────────┬─────────────────────────────┤
│  Short-term     │  Long-term      │     Course Catalog          │
│  Memory         │  Memory         │     (Vector Search)         │
│  (Checkpointer) │  (Vector Store) │                             │
└─────────────────┴─────────────────┴─────────────────────────────┘
```

**System Architecture Diagram Description**: The diagram shows three connected components at the top: User Input (CLI/API) connects to LangGraph Agent, which connects to OpenAI GPT (LLM). Below these, the Redis Context Engine contains three sub-components: Short-term Memory (Checkpointer), Long-term Memory (Vector Store), and Course Catalog (Vector Search).

### Key Components

1. **LangGraph Agent**: Orchestrates the conversation flow and decision-making
2. **Redis Context Engine**: Manages all context and memory operations
3. **OpenAI Integration**: Provides language understanding and generation
4. **Tool System**: Enables the agent to search, recommend, and remember
5. **CLI Interface**: Provides an interactive way to chat with the agent

## Environment Setup

Let's set up our environment to explore the project:

In [ ]:
# Install the Redis Context Course package
%pip install --upgrade -q -e ../../reference-agent

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up environment with consistent defaults
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Non-interactive check for OpenAI key
if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY is not set. Some examples will use mock data.")
    print("   See the setup notebook for configuration instructions.")
else:
    print("✅ Environment configured successfully")
    print(f"   Redis URL: {REDIS_URL}")
    print(f"   Agent Memory URL: {AGENT_MEMORY_URL}")

## Core Features

Let's explore the key features our agent provides:

### Feature 1: Intelligent Course Search

The agent can search through course catalogs using both semantic and structured search:

- **Semantic vector search** using OpenAI embeddings with RedisVL
- **Structured filters** (department, difficulty, format)
- **Hybrid search** and relevance ranking

**Example Usage:**
```python
from redis_context_course.course_manager import CourseManager
course_manager = CourseManager()

# Run a semantic search
results = await course_manager.search_courses("machine learning", limit=3)
for course in results:
    print(f"{course.course_code}: {course.title}")
```

### Feature 2: Personalized Recommendations

The agent provides personalized course recommendations based on student profiles and preferences:

- **Combines** interests, history, prerequisites, and preferences
- **Ranks courses** and explains each recommendation
- **Considers** academic progress and requirements

**Example Usage:**
```python
from redis_context_course.models import StudentProfile, CourseFormat, DifficultyLevel

profile = StudentProfile(
    name="Alex Johnson", 
    major="Computer Science", 
    year=2,
    completed_courses=["CS101", "MATH101"],
    interests=["machine learning", "web development"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE
)

recommendations = await course_manager.get_recommendations(profile, limit=3)
```

### Feature 3: Persistent Memory System

The agent remembers student interactions and builds context over time:

- **Stores** preferences, goals, experiences, and key conversation summaries
- **Supports** store, retrieve, consolidate, update, and expire operations
- **Uses** Agent Memory Server for sophisticated memory management

**Example Usage:**
```python
from agent_memory_client import MemoryAPIClient, MemoryClientConfig

config = MemoryClientConfig(
    base_url="http://localhost:8088", 
    default_namespace="redis_university"
)
memory_client = MemoryAPIClient(config=config)

# Store a preference
await memory_client.create_long_term_memory([
    ClientMemoryRecord(
        text="Student prefers online courses due to work schedule",
        memory_type=MemoryTypeEnum.SEMANTIC
    )
])

# Search memories
results = await memory_client.search_long_term_memory(
    text="online courses", limit=3
)
```

### Feature 4: LangGraph Workflow

The agent uses LangGraph for sophisticated workflow orchestration:

```
┌─────────────────┐
│  User Input     │
└─────────┬───────┘
          │
          ▼
┌─────────────────┐   (retrieve context)
│ Retrieve        │◄────────────────────
│ Context         │
└─────────┬───────┘
          │
          ▼
┌─────────────────┐   (uses tools when needed)
│ Agent Reasoning │
└─────────┬───────┘
          │
          ▼
┌─────────────────┐   (checkpointer + long-term)
│ Store Memory    │
└─────────────────┘
```

**Available Tools**: search courses, get recommendations, store preferences/goals, fetch student context.

### Feature 5: Interactive CLI Interface

The agent provides a rich command-line interface for easy interaction:

- **Rich formatting**, history, and help
- **Typing indicators**, markdown rendering, friendly errors
- **Session persistence** and conversation continuity

**Example Session:**
```text
You: I'm interested in machine learning courses
Agent: Great! I found several ML courses that match your interests.
        Based on your CS major, I recommend:
        • CS401: Machine Learning Fundamentals
        • CS402: Deep Learning Applications

You: I prefer online courses
Agent: Perfect! Both courses offer online options. I'll remember 
       your preference for future recommendations.
```

## ✅ Knowledge Check: Project Architecture

**Question 1**: Which component orchestrates the conversation flow in our agent?
- [ ] Redis Context Engine
- [ ] LangGraph Agent
- [ ] OpenAI GPT
- [ ] CLI Interface

**Question 2**: What are the three main parts of the Redis Context Engine?
- [ ] Input, Processing, Output
- [ ] Short-term Memory, Long-term Memory, Course Catalog
- [ ] Search, Recommend, Remember
- [ ] System, User, Domain

**Question 3**: What type of search does the course catalog use?
- [ ] Keyword search only
- [ ] SQL database queries
- [ ] Vector search with embeddings
- [ ] Regular expressions

*Answers: 1-B, 2-B, 3-C*

## Technical Implementation

Let's examine the technical stack and implementation details:

### Technology Stack

**AI/ML Components:**
- **OpenAI GPT** for natural language generation
- **text-embedding-3-small** for vector embeddings
- **LangChain + LangGraph** for agent orchestration

**Data & Storage:**
- **Redis 8** for vectors and metadata storage
- **RedisVL** for vector search operations
- **LangGraph checkpointing** in Redis for conversation state
- **Agent Memory Server** for sophisticated memory management

**Development:**
- **Python 3.10+** with modern async/await patterns
- **Pydantic** for data validation and serialization
- **Rich/Click** for beautiful CLI interfaces
- **asyncio** for concurrent operations

**Quality & Testing:**
- **Pytest** for comprehensive testing
- **Black, isort** for code formatting
- **MyPy** for type checking

### Architecture Patterns

**Repository Pattern:**
- Isolate data access (CourseManager, MemoryClient)
- Clean separation between business logic and data storage

**Strategy Pattern:**
- Multiple search/retrieval strategies (semantic, keyword, hybrid)
- Pluggable memory extraction strategies

**Observer Pattern:**
- State persistence & consolidation via Redis checkpointer
- Automatic memory extraction triggers

**Factory Pattern:**
- Constructors for memories and course artifacts
- Tool creation and configuration

### Performance Characteristics

**Response Times:**
- Redis operations: <1ms
- Vector search: <50ms
- Memory retrieval: <100ms
- End-to-end response: <2s

**Scalability:**
- Scales horizontally with Redis clustering
- Stateless workers for high availability
- Efficient memory usage with vector compression

## 🛠️ Hands-on Exercise: Explore the Codebase

**Task**: Navigate the reference agent codebase and answer these questions:

1. **Structure**: What are the main modules in `redis_context_course/`?
2. **Models**: What data models are defined in `models.py`?
3. **Tools**: What tools are available in `tools.py`?
4. **Examples**: What examples are provided in the `examples/` directory?

**Expected Time**: 10 minutes  
**Deliverable**: Written exploration notes

### Your Exploration Notes:
*(Write your findings here or in a separate document)*

1. **Main Modules**: 

2. **Data Models**: 

3. **Available Tools**: 

4. **Examples**: 

## Getting Started Guide

Here's how to set up and run the Redis University Class Agent:

### Prerequisites
- Python 3.10+
- Docker and Docker Compose
- OpenAI API key

### Quick Setup
1. **Navigate to the reference agent directory**
   ```bash
   cd python-recipes/context-engineering/reference-agent
   ```

2. **Install dependencies**
   ```bash
   pip install -e .
   ```

3. **Start services with Docker Compose**
   ```bash
   cd .. # Back to context-engineering directory
   docker-compose up -d
   ```

4. **Configure environment**
   ```bash
   cp .env.example .env
   # Edit .env to set OPENAI_API_KEY
   ```

5. **Generate and ingest sample data**
   ```bash
   cd reference-agent
   python -m redis_context_course.scripts.generate_courses
   python -m redis_context_course.scripts.ingest_courses
   ```

6. **Start the agent**
   ```bash
   python -m redis_context_course.cli --student-id your_name
   ```

### Verification Steps
- ✅ Redis connection reports "Healthy"
- ✅ Course catalog shows 50+ courses
- ✅ Agent greets you and can search for "programming"
- ✅ Preferences persist across messages

**Note**: Complete setup instructions are provided in the next notebook: `03_setup_environment.ipynb`

## Learning Objectives Summary

By working with this project throughout the course, you'll learn:

### Core Concepts
- **Context engineering principles** and patterns
- **Agent workflow design** and tool integration
- **Memory modeling** (short-term, long-term, consolidation)
- **Vector search** and retrieval strategies

### Technical Skills
- **Designing context-aware agents** with LangGraph
- **Using Redis 8 and RedisVL** for vector search and state management
- **Building and evaluating** retrieval and memory strategies
- **Performance tuning** for production systems

### Best Practices
- **Error handling** and robustness patterns
- **Persistence** and state management
- **Observability** and debugging techniques
- **Scalability** considerations

## Course Roadmap

Here's what we'll cover in the upcoming sections:

### Section 1: Introduction (Current)
- ✅ What is Context Engineering?
- ✅ Project Overview: Redis University Class Agent
- 🔄 Environment Setup (Next)

### Section 2: Setting up System Context
- System instructions and prompts
- Defining available tools
- Tool selection strategies

### Section 3: Memory Management
- Working memory with extraction strategies
- Long-term memory and integration
- Memory tools and LLM control

### Section 4: Optimizations
- Context window management
- Retrieval strategies and grounding
- Tool optimization
- Crafting data for LLMs

## 🤔 Reflection: Architecture Insights

Think about the architecture we've explored:

1. **Component Separation**: Why is it beneficial to separate LangGraph, Redis, and OpenAI?
2. **Memory Types**: How do short-term and long-term memory serve different purposes?
3. **Tool System**: What advantages does the tool-based approach provide?
4. **Scalability**: How would this architecture handle thousands of concurrent users?

**Consider discussing these questions with fellow learners or in the course forum.**

---

## Conclusion

The Redis University Class Agent represents a comprehensive example of context engineering in practice. It demonstrates how to build intelligent, context-aware AI systems that can:

- **Remember and learn** from user interactions
- **Provide personalized experiences** based on individual needs
- **Scale efficiently** using Redis as the context engine
- **Integrate seamlessly** with modern AI frameworks
- **Maintain consistency** across multiple sessions and conversations

As we progress through this course, you'll gain hands-on experience with each component of the system, learning not just how to build context-aware AI agents, but understanding the principles and patterns that make them effective.

## Ready to Continue?

Now that you understand the project overview and architecture, you're ready to set up your development environment. In **03_setup_environment.ipynb**, we'll cover:

- Complete environment configuration
- Service setup and verification
- Troubleshooting common issues
- Running your first agent interactions

After that, we'll dive into **Section 2: Setting up System Context** to explore:

- How to define what your AI agent should know about itself
- Techniques for crafting effective system prompts
- Methods for defining and managing agent tools
- Best practices for setting capability boundaries

Let's continue building your expertise in context engineering! 🚀